In [1]:
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.options.display.max_columns = 110


In [2]:
byCategory=pd.read_csv('0614byCategory.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
##將檔期時間調整為至少一個月(檔期開始時間最晚為2019-03-30)
byCategory=byCategory[byCategory.business_hour_order_time_s<'2019-03-30']

In [4]:
##找出檔期長度超過3年,檔期長度即設為1100天
byCategory.loc[byCategory.campagin_period>1100,'campagin_period']=1100

In [5]:
byCategory=byCategory[['business_hour_guid','product_launch_month', 'product_launch_hour',
       'product_launch_weekday','code', 'parent_code', 'is_Presales', 'is_Longtern',  
        'enable_isp','is_Discount','installment_3months', 'installment_6months', 'installment_12months',
       'deny_installment', 
       'item_orig_price', 'item_price','item_default_daily_amount',
       'quantity_multiplier','shippingdate', 'freight_amount',
        'campagin_period', 'order_deliver_days','unit_price',
       'category_unit_price', 'compare_cate_unit_price',
       'guid_subtotal', 'quantity_sold_guid', 'subtotal_allguid',
       'quantity_sold_allguid', 'avg_unit_price', 'price_gap',
        'hottimes']]


In [6]:
byCategory['unit_price']=byCategory['item_price']/byCategory['quantity_multiplier']

In [7]:
byCategory.columns

Index(['business_hour_guid', 'product_launch_month', 'product_launch_hour',
       'product_launch_weekday', 'code', 'parent_code', 'is_Presales',
       'is_Longtern', 'enable_isp', 'is_Discount', 'installment_3months',
       'installment_6months', 'installment_12months', 'deny_installment',
       'item_orig_price', 'item_price', 'item_default_daily_amount',
       'quantity_multiplier', 'shippingdate', 'freight_amount',
       'campagin_period', 'order_deliver_days', 'unit_price',
       'category_unit_price', 'compare_cate_unit_price', 'guid_subtotal',
       'quantity_sold_guid', 'subtotal_allguid', 'quantity_sold_allguid',
       'avg_unit_price', 'price_gap', 'hottimes'],
      dtype='object')

In [8]:
df=byCategory
df.shape

(224074, 32)

In [9]:
#發現 code,category_unit_price,compare_cate_unit_price 有nan值 因為分類關係
df.code=df.code.fillna(0)
df.shippingdate=df.shippingdate.fillna(0)
df.isna().sum()

business_hour_guid              0
product_launch_month            0
product_launch_hour             0
product_launch_weekday          0
code                            0
parent_code                     0
is_Presales                     0
is_Longtern                     0
enable_isp                      0
is_Discount                     0
installment_3months             0
installment_6months             0
installment_12months            0
deny_installment                0
item_orig_price                 0
item_price                      0
item_default_daily_amount       0
quantity_multiplier             0
shippingdate                    0
freight_amount                  0
campagin_period                 0
order_deliver_days              0
unit_price                      0
category_unit_price          3528
compare_cate_unit_price      3528
guid_subtotal                   0
quantity_sold_guid              0
subtotal_allguid                0
quantity_sold_allguid           0
avg_unit_price

In [10]:
df['category_unit_price']=df['category_unit_price'].fillna(df['unit_price'])

In [11]:
df['compare_cate_unit_price']=df['compare_cate_unit_price'].fillna(1)

## 將檔案順序打散

In [12]:
df=df.sample(frac=1, replace=False, random_state=1)

In [13]:
df=df.reset_index(drop=True)

將資料分為輸入變數與目標變數

In [14]:
target=pd.read_csv('target.csv')
code=pd.read_csv('code_time.csv')
target.head(3)

,business_hour_guid,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days
0,00000793-A001-4C8C-9196-3E6971181246,399.0,399.0,1.0,399.0,1.0,399.0,1.0,399.0,1.0,399.0,1.0
1,00001911-C3DF-4143-BC25-8AA5FA1A9AF4,1150.0,1150.0,1.0,1150.0,1.0,1150.0,1.0,1150.0,1.0,1150.0,1.0
2,00001928-AE75-4AA9-8A47-4BDE96D08C2D,23040.0,0.0,0.0,11520.0,0.5,23040.0,1.0,23040.0,1.0,23040.0,1.0


In [15]:
df=pd.merge(df,target,how='left')
df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,code,parent_code,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,536.0,536.0,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.200,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0
1,F0AB8354-EB2A-44F6-87D7-F71DAFFF1D28,6,7,3,166.0,166.0,0,0,0,1,0,0,0,0,890.0,239.0,20,1.0,0.0,0.0,13,0,239.0,359.0,0.666,4302.0,18.0,0.0,0.0,239.0,651.0,1,4302.0,2390.0,0.556,4302.0,1.000,4302.0,1.0,4302.0,1.0,4302.0,1.0
2,D7EE78D5-DE57-4486-881C-1DF8D8B220BC,9,7,2,1604.0,166.0,0,0,0,1,1,0,0,0,799.0,429.0,20,1.0,7.0,0.0,21,0,429.0,319.0,1.345,2574.0,6.0,0.0,0.0,429.0,370.0,0,2574.0,1287.0,0.500,1287.0,0.500,2574.0,1.0,2574.0,1.0,2574.0,1.0


In [16]:
df=pd.merge(df,code)
df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,code,parent_code,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days,code_total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days,code_max_7days,code_min_7days,code_median_7days,code_mean_7days,code_max_14days,code_min_14days,code_median_14days,code_mean_14days,code_max_30days,code_min_30days,code_median_30days,code_mean_30days,code_max_60days,code_min_60days,code_median_60days,code_mean_60days,code_max_180days,code_min_180days,code_median_180days,code_mean_180days
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,536.0,536.0,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.2,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0
1,9584B6E4-264C-4FFC-8EF6-5223301E692A,9,7,4,536.0,536.0,0,1,0,1,1,0,0,0,1380.0,799.0,20,1.0,0.0,0.0,81,0,799.0,299.0,2.672,3995.0,5.0,0.0,0.0,799.0,581.0,0,3995.0,799.0,0.2,799.0,0.200,799.0,0.2,3196.0,0.8,3995.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0
2,124C1F67-488B-4B55-B449-C45650BDECC7,4,7,1,536.0,536.0,0,0,0,1,1,1,1,0,1718.0,1150.0,20,1.0,0.0,0.0,21,0,1150.0,299.0,3.846,1150.0,1.0,0.0,0.0,1150.0,568.0,0,1150.0,1150.0,1.0,1150.0,1.000,1150.0,1.0,1150.0,1.0,1150.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0


In [17]:
parent_code=code
parent_code.columns=['parent_code', 'parent_code_total_sales_amount', 'parent_code_sales_7days', 'parent_code_ratio_7days',
       'parent_code_sales_14days', 'parent_code_ratio_14days', 'parent_code_sales_30days',
       'parent_code_ratio_30days', 'parent_code_sales_60days', 'parent_code_ratio_60days',
       'parent_code_sales_180days', 'parent_code_ratio_180days', 'parent_code_max_7days',
       'parent_code_min_7days', 'parent_code_median_7days', 'parent_code_mean_7days',
       'parent_code_max_14days', 'parent_code_min_14days', 'parent_code_median_14days',
       'parent_code_mean_14days', 'parent_code_max_30days', 'parent_code_min_30days',
       'parent_code_median_30days', 'parent_code_mean_30days', 'parent_code_max_60days',
       'parent_code_min_60days', 'parent_code_median_60days', 'parent_code_mean_60days',
       'parent_code_max_180days', 'parent_code_min_180days', 'parent_code_median_180days',
       'parent_code_mean_180days']
parent_code.head(3)

,parent_code,parent_code_total_sales_amount,parent_code_sales_7days,parent_code_ratio_7days,parent_code_sales_14days,parent_code_ratio_14days,parent_code_sales_30days,parent_code_ratio_30days,parent_code_sales_60days,parent_code_ratio_60days,parent_code_sales_180days,parent_code_ratio_180days,parent_code_max_7days,parent_code_min_7days,parent_code_median_7days,parent_code_mean_7days,parent_code_max_14days,parent_code_min_14days,parent_code_median_14days,parent_code_mean_14days,parent_code_max_30days,parent_code_min_30days,parent_code_median_30days,parent_code_mean_30days,parent_code_max_60days,parent_code_min_60days,parent_code_median_60days,parent_code_mean_60days,parent_code_max_180days,parent_code_min_180days,parent_code_median_180days,parent_code_mean_180days
0,0.0,18506893.0,5865675.0,0.317,8728702.0,0.472,12569867.0,0.679,15237826.0,0.823,17362861.0,0.938,9999.0,100.0,1028.524461,899.0,9999.0,100.0,1043.728566,899.0,16990.0,100.0,1103.200544,950.0,16990.0,100.0,1132.418698,989.0,16990.0,100.0,1140.118261,990.0
1,2.0,1515784.0,684725.0,0.452,970749.0,0.640,1187436.0,0.783,1247623.0,0.823,1382325.0,0.912,4038.0,149.0,541.712816,449.0,8076.0,149.0,555.030875,458.0,8076.0,99.0,560.375649,458.0,8076.0,99.0,561.739307,458.0,8076.0,99.0,569.326606,458.0
2,4.0,44808.0,1851.0,0.041,2631.0,0.059,4509.0,0.101,14144.0,0.316,32940.0,0.735,858.0,490.0,617.000000,503.0,858.0,390.0,526.200000,490.0,939.0,390.0,644.142857,503.0,1878.0,390.0,884.000000,898.5,1878.0,230.0,968.823529,939.0


In [18]:
df=pd.merge(df,parent_code)
df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,code,parent_code,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days,code_total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days,code_max_7days,code_min_7days,code_median_7days,code_mean_7days,code_max_14days,code_min_14days,code_median_14days,code_mean_14days,code_max_30days,code_min_30days,code_median_30days,code_mean_30days,code_max_60days,code_min_60days,code_median_60days,code_mean_60days,code_max_180days,code_min_180days,code_median_180days,code_mean_180days,parent_code_total_sales_amount,parent_code_sales_7days,parent_code_ratio_7days,parent_code_sales_14days,parent_code_ratio_14days,parent_code_sales_30days,parent_code_ratio_30days,parent_code_sales_60days,parent_code_ratio_60days,parent_code_sales_180days,parent_code_ratio_180days,parent_code_max_7days,parent_code_min_7days,parent_code_median_7days,parent_code_mean_7days,parent_code_max_14days,parent_code_min_14days,parent_code_median_14days,parent_code_mean_14days,parent_code_max_30days,parent_code_min_30days,parent_code_median_30days,parent_code_mean_30days,parent_code_max_60days,parent_code_min_60days,parent_code_median_60days,parent_code_mean_60days,parent_code_max_180days,parent_code_min_180days,parent_code_median_180days,parent_code_mean_180days
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,536.0,536.0,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.2,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0
1,9584B6E4-264C-4FFC-8EF6-5223301E692A,9,7,4,536.0,536.0,0,1,0,1,1,0,0,0,1380.0,799.0,20,1.0,0.0,0.0,81,0,799.0,299.0,2.672,3995.0,5.0,0.0,0.0,799.0,581.0,0,3995.0,799.0,0.2,799.0,0.200,799.0,0.2,3196.0,0.8,3995.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0
2,124C1F67-488B-4B55-B449-C45650BDECC7,4,7,1,536.0,536.0,0,0,0,1,1,1,1,0,1718.0,1150.0,20,1.0,0.0,0.0,21,0,1150.0,299.0,3.846,1150.0,1.0,0.0,0.0,1150.0,568.0,0,1150.0,1150.0,1.0,1150.0,1.000,1150.0,1.0,1150.0,1.0,1150.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946

## 觀察相關係數

In [ ]:
corrtable=(df.corr()[df.corr().index=='sales_7days']).T
sales_7days=corrtable.sort_values(ascending=False,by='sales_7days')
sales_7days=sales_7days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_7days

In [ ]:
corrtable=(df.corr()[df.corr().index=='sales_14days']).T
sales_14days=corrtable.sort_values(ascending=False,by='sales_14days')
sales_14days=sales_14days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_14days

In [ ]:
corrtable=(df.corr()[df.corr().index=='sales_30days']).T
sales_30days=corrtable.sort_values(ascending=False,by='sales_30days')
sales_30days=sales_30days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_30days

In [ ]:
corrtable=(df.corr()[df.corr().index=='sales_60days']).T
sales_60days=corrtable.sort_values(ascending=False,by='sales_60days')
sales_60days=sales_60days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_60days

In [ ]:
corrtable=(df.corr()[df.corr().index=='sales_180days']).T
sales_180days=corrtable.sort_values(ascending=False,by='sales_180days')
sales_180days=sales_180days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_180days

## 因為code的類別數較大,故使用計數編碼

In [19]:
parent_code=df.parent_code.value_counts().to_frame()
parent_code=parent_code.rename(columns={'parent_code':'parent_code_counts'})
parent_code.index.name='parent_code'
#code.columns=[['code','code_counts']]
parent_code=parent_code.reset_index(drop=False)
df=pd.merge(df,parent_code,how='left')
df=df.drop(columns=['parent_code'])

df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,code,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days,code_total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days,code_max_7days,code_min_7days,code_median_7days,code_mean_7days,code_max_14days,code_min_14days,code_median_14days,code_mean_14days,code_max_30days,code_min_30days,code_median_30days,code_mean_30days,code_max_60days,code_min_60days,code_median_60days,code_mean_60days,code_max_180days,code_min_180days,code_median_180days,code_mean_180days,parent_code_total_sales_amount,parent_code_sales_7days,parent_code_ratio_7days,parent_code_sales_14days,parent_code_ratio_14days,parent_code_sales_30days,parent_code_ratio_30days,parent_code_sales_60days,parent_code_ratio_60days,parent_code_sales_180days,parent_code_ratio_180days,parent_code_max_7days,parent_code_min_7days,parent_code_median_7days,parent_code_mean_7days,parent_code_max_14days,parent_code_min_14days,parent_code_median_14days,parent_code_mean_14days,parent_code_max_30days,parent_code_min_30days,parent_code_median_30days,parent_code_mean_30days,parent_code_max_60days,parent_code_min_60days,parent_code_median_60days,parent_code_mean_60days,parent_code_max_180days,parent_code_min_180days,parent_code_median_180days,parent_code_mean_180days,parent_code_counts
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,536.0,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.2,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,52043
1,9584B6E4-264C-4FFC-8EF6-5223301E692A,9,7,4,536.0,0,1,0,1,1,0,0,0,1380.0,799.0,20,1.0,0.0,0.0,81,0,799.0,299.0,2.672,3995.0,5.0,0.0,0.0,799.0,581.0,0,3995.0,799.0,0.2,799.0,0.200,799.0,0.2,3196.0,0.8,3995.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,52043
2,124C1F67-488B-4B55-B449-C45650BDECC7,4,7,1,536.0,0,0,0,1,1,1,1,0,1718.0,1150.0,20,1.0,0.0,0.0,21,0,1150.0,299.0,3.846,1150.0,1.0,0.0,0.0,1150.0,568.0,0,1150.0,1150.0,1.0,1150.0,1.000,1150.0,1.0,1150.0,1.0,1150.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.94

In [20]:
code=df.code.value_counts().to_frame()
code=code.rename(columns={'code':'code_counts'})
code.index.name='code'
#code.columns=[['code','code_counts']]
code=code.reset_index(drop=False)
df=pd.merge(df,code,how='left')

df=df.drop(columns=['code'])
df.head()

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days,code_total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days,code_max_7days,code_min_7days,code_median_7days,code_mean_7days,code_max_14days,code_min_14days,code_median_14days,code_mean_14days,code_max_30days,code_min_30days,code_median_30days,code_mean_30days,code_max_60days,code_min_60days,code_median_60days,code_mean_60days,code_max_180days,code_min_180days,code_median_180days,code_mean_180days,parent_code_total_sales_amount,parent_code_sales_7days,parent_code_ratio_7days,parent_code_sales_14days,parent_code_ratio_14days,parent_code_sales_30days,parent_code_ratio_30days,parent_code_sales_60days,parent_code_ratio_60days,parent_code_sales_180days,parent_code_ratio_180days,parent_code_max_7days,parent_code_min_7days,parent_code_median_7days,parent_code_mean_7days,parent_code_max_14days,parent_code_min_14days,parent_code_median_14days,parent_code_mean_14days,parent_code_max_30days,parent_code_min_30days,parent_code_median_30days,parent_code_mean_30days,parent_code_max_60days,parent_code_min_60days,parent_code_median_60days,parent_code_mean_60days,parent_code_max_180days,parent_code_min_180days,parent_code_median_180days,parent_code_mean_180days,parent_code_counts,code_counts
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.2,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,52043,28492
1,9584B6E4-264C-4FFC-8EF6-5223301E692A,9,7,4,0,1,0,1,1,0,0,0,1380.0,799.0,20,1.0,0.0,0.0,81,0,799.0,299.0,2.672,3995.0,5.0,0.0,0.0,799.0,581.0,0,3995.0,799.0,0.2,799.0,0.200,799.0,0.2,3196.0,0.8,3995.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,52043,28492
2,124C1F67-488B-4B55-B449-C45650BDECC7,4,7,1,0,0,0,1,1,1,1,0,1718.0,1150.0,20,1.0,0.0,0.0,21,0,1150.0,299.0,3.846,1150.0,1.0,0.0,0.0,1150.0,568.0,0,1150.0,1150.0,1.0,1150.0,1.000,1150.0,1.0,1150.0,1.0,1150.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.9

In [21]:
df['orig_unit_price']=df['item_orig_price']/df['quantity_multiplier']

## 預測銷量

In [ ]:
df.to_csv('final_df.csv',index=False)

In [2]:
final_df=pd.read_csv('final_df.csv')

In [7]:
#目標變數
target=['sales_7days','ratio_7days','sales_14days','ratio_14days','sales_30days','ratio_30days','sales_60days',
 'ratio_60days','sales_180days','ratio_180days','guid_subtotal']
#特徵變數
features=['business_hour_guid','product_launch_month','product_launch_hour','product_launch_weekday',
 'is_Presales','is_Longtern','enable_isp','is_Discount','installment_3months','installment_6months',
 'installment_12months','item_orig_price','item_price','item_default_daily_amount',
 'quantity_multiplier','freight_amount',#'campagin_period', 
 'orig_unit_price', 'unit_price','category_unit_price','compare_cate_unit_price',
          'subtotal_allguid','quantity_sold_allguid',#舊商品新檔期
 'avg_unit_price','price_gap','hottimes',
 'code_total_sales_amount','code_sales_7days','code_ratio_7days','code_sales_14days','code_ratio_14days',
 'code_sales_30days','code_ratio_30days', 'code_sales_60days','code_ratio_60days','code_sales_180days',
 'code_ratio_180days','code_max_7days','code_min_7days','code_median_7days','code_mean_7days',
 'code_max_14days','code_min_14days','code_median_14days','code_mean_14days','code_max_30days','code_min_30days',
 'code_median_30days', 'code_mean_30days', 'code_max_60days', 'code_min_60days',
 'code_median_60days', 'code_mean_60days', 'code_max_180days', 'code_min_180days',
 'code_median_180days', 'code_mean_180days', 'parent_code_total_sales_amount',
 'parent_code_sales_7days', 'parent_code_ratio_7days', 'parent_code_sales_14days', 'parent_code_ratio_14days',
 'parent_code_sales_30days', 'parent_code_ratio_30days', 'parent_code_sales_60days', 'parent_code_ratio_60days',
 'parent_code_sales_180days', 'parent_code_ratio_180days', 'parent_code_max_7days','parent_code_min_7days',
 'parent_code_median_7days','parent_code_mean_7days', 'parent_code_max_14days', 'parent_code_min_14days',
 'parent_code_median_14days','parent_code_mean_14days', 'parent_code_max_30days', 'parent_code_min_30days',
 'parent_code_median_30days', 'parent_code_mean_30days', 'parent_code_max_60days', 'parent_code_min_60days',
 'parent_code_median_60days', 'parent_code_mean_60days', 'parent_code_max_180days', 'parent_code_min_180days',
 'parent_code_median_180days', 'parent_code_mean_180days', 'parent_code_counts', 'code_counts']

In [9]:
final_df[target]

,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days,guid_subtotal
0,3264.0,0.200,11968.0,0.733,16320.0,1.000,16320.0,1.000,16320.0,1.000,16320.0
1,799.0,0.200,799.0,0.200,799.0,0.200,3196.0,0.800,3995.0,1.000,3995.0
2,1150.0,1.000,1150.0,1.000,1150.0,1.000,1150.0,1.000,1150.0,1.000,1150.0
3,498.0,1.000,498.0,1.000,498.0,1.000,498.0,1.000,498.0,1.000,498.0
4,0.0,0.000,370.0,0.333,1110.0,1.000,1110.0,1.000,1110.0,1.000,1110.0
5,449.0,0.143,1347.0,0.429,1347.0,0.429,2694.0,0.857,3143.0,1.000,3143.0
6,639.0,0.500,639.0,0.500,1278.0,1.000,1278.0,1.000,1278.0,1.000,1278.0
7,1280.0,0.333,3840.0,1.000,3840.0,1.000,3840.0,1.000,3840.0,1.000,3840.0
8,248.0,1.000,248.0,1.000,248.0,1.000,248.0,1.000,248.0,1.000,248.0
9,0.0,0.000,0.0,0.000,378.0,0.500,756.0,1.000,756.0,1.000,756.0


In [23]:
##將銷量相關取log
df_new=df.drop(columns='business_hour_guid')
ohe_df=pd.get_dummies(df_new,columns=categoryIndex)
ohe_df[continousIndex] = np.log1p(ohe_df[continousIndex])
ohe_df[targetIndex] = np.log1p(ohe_df[targetIndex])

In [8]:
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
def trainTestValSplit(df):
    # 將資料切分為訓練(80%),測試(10%)和驗證(10%)三份。
    train=df.sample(frac=0.8,random_state=0)
    test=df.drop(train.index )
    val=test.sample(frac=0.5,random_state=0)
    test=test.drop(val.index)
    return train,test,val

def dfXYSplit(df,targetIndex,targetName):
    # 將特徵和目標變數切成兩份資料。
    
    dfX=df.drop(columns=targetIndex)
    dfY=df[targetName]
    
    return dfX,dfY
def trainmodel(df,targetIndex,targetname,model):
    train,test,val=trainTestValSplit(df)
    trainX,trainY=dfXYSplit(train,targetIndex,targetname)
    testX,testY=dfXYSplit(test,targetIndex,targetname)
    valX,valY=dfXYSplit(val,targetIndex,targetname)
    model.fit(trainX,trainY)
    testPredY=model.predict(testX)
    valPredY=model.predict(valX)
    print(model)
    print("val_mean squared error: %.2f"
      % mean_squared_error(valY, valPredY))

    print("val_mean_absolute_error: %.2f"
      % mean_absolute_error(valY, valPredY))
    print("val_r2_score: %.2f"
      % r2_score(valY,valPredY))
    subtotal=pd.DataFrame()
    subtotal['predict']=np.expm1(np.array(testPredY.tolist()).reshape(-1))
    subtotal['real']=np.expm1(testY[testY.columns[0]].reset_index(drop=True))
    def check(x):
        if x <1 :
            return 0
        else:
            return x
    subtotal.predict=subtotal.predict.apply(check)

    print("test_mean squared error: %.2f"
      % mean_squared_error(subtotal['real'],subtotal['predict']))

    print("test_mean_absolute_error: %.2f"
      % mean_absolute_error(subtotal['real'],subtotal['predict']))
    print("test_r2_score: %.2f"
      % r2_score(subtotal['real'],subtotal['predict']))
    return subtotal

## DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=0)
targetname=['sales_7days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

In [ ]:
model = DecisionTreeRegressor(random_state=0)
targetname=['sales_14days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

In [ ]:
model = DecisionTreeRegressor(random_state=0)
targetname=['sales_30days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

In [ ]:
model = DecisionTreeRegressor(random_state=0)
targetname=['sales_60days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

In [ ]:
model = DecisionTreeRegressor(random_state=0)
targetname=['sales_180days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

### XGBoost

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
targetname=['sales_7days']
xgb=trainmodel(ohe_df,targetIndex,targetname,model)
xgb.head(6)

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
targetname=['sales_14days']
xgb=trainmodel(ohe_df,targetIndex,targetname,model)
xgb.head(6)

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
targetname=['sales_30days']
xgb=trainmodel(ohe_df,targetIndex,targetname,model)
xgb.head(6)

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
targetname=['sales_60days']
xgb=trainmodel(ohe_df,targetIndex,targetname,model)
xgb.head(6)

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
targetname=['sales_180days']
xgb=trainmodel(ohe_df,targetIndex,targetname,model)
xgb.head(6)

## RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(max_depth=2, random_state=0,n_estimators=100)
targetname=['sales_7days']
rf=trainmodel(ohe_df,targetIndex,targetname,model)
rf.head(6)

## LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
targetname=['sales_7days']
lr=trainmodel(ohe_df,targetIndex,targetname,model)
lr.head(6)

## Ridge 

In [ ]:
from sklearn.linear_model import Lasso, Ridge
model=Ridge()
targetname=['sales_7days']
rl=trainmodel(ohe_df,targetIndex,targetname,model)
rl.head(6)

## 將資料切分訓練(80%),測試(10%)和驗證(10%)三份


* 未作細調參數前的MAE R2整理：

Model                    | Val_MAE                  |Val_R2                  | 
----------------         |:---------:               |---------:              | 
DecisionTreeRegressor    | 0.01/0.01/0.00/0.00/0.00 |1.0/1.0/1.0/1.0/1.0     |    0 | 
XGBRegressor             | 0.04/0.04/0.03/0.02/0.02 |1.0/1.0/1.0/1.0/1.0     |    0 |  
RandomForestRegressor    | 0.44                     |0.96                    |    0 |  
LinearRegression         | 1.94                     |0.62                    |    0 |  
Ridge                    | 1.93                     |0.62                    |    0 |  


* 因為所有數值型變數都有經過對數轉換，故在檢查測試集還原log後的表現，選擇以DecisionTree 與XGBregressor 進行參數調教

## 使用網格搜尋法
* 將資料重新分為訓練集與測試集

In [ ]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
targetname=['sales_7days']
trainX,trainY=dfXYSplit(ohe_df,targetIndex,targetname)
x_train, x_test, y_train, y_test = train_test_split(trainX,trainY, test_size=0.2, random_state=4)

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
model =  xgb.XGBRegressor( )


# 設定要訓練的超參數組合
n_estimators = [10,25,50, 100, 150]
max_depth = [1, 3, 5,7,10]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(model, param_grid,n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train,y_train)
print("Best Combination: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


In [ ]:
from sklearn.tree import DecisionTreeRegressor

model =  DecisionTreeRegressor()

# 設定要訓練的超參數組合
min_samples_split=[2,4,6,8,10]
min_samples_leaf=[1, 3,5,7,9]
max_depth = [1,3, 5,10,15,20]
param_grid = dict(max_depth=max_depth,min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(model, param_grid,n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train,y_train)
print("Best combination: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


In [ ]:

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
model = DecisionTreeRegressor(random_state=0,max_depth=20,min_samples_leaf=3,min_samples_split=8)
targetname=['sales_7days','sales_14days','sales_30days','sales_60days','sales_180days']
trainX,trainY=dfXYSplit(ohe_df,targetIndex,targetname)
X_train, X_test, y_train, y_test = train_test_split(trainX,trainY, test_size=0.2, random_state=4)
model.fit(X_train,y_train)
y_predict=model.predict(X_test)


#log 
print("test_mean squared error: %.2f"
      % mean_squared_error(y_test,y_predict))
print("test_mean_absolute_error: %.2f"
      % mean_absolute_error(y_test,y_predict))
print("test_r2_score: %.2f"
      % r2_score(y_test,y_predict))

### DecisionTree_Final Output

In [ ]:
targetname=['sales_7days','sales_14days','sales_30days','sales_60days','sales_180days']
train,test,val=trainTestValSplit(ohe_df)
trainX,trainY=dfXYSplit(train,targetIndex,targetname)
testX,y_test=dfXYSplit(test,targetIndex,targetname)
valX,valY=dfXYSplit(val,targetIndex,targetname)

In [ ]:
model = DecisionTreeRegressor(random_state=0)
#model = DecisionTreeRegressor(random_state=0,max_depth=20,min_samples_leaf=3,min_samples_split=8)
targetname=['sales_7days','sales_14days','sales_30days','sales_60days','sales_180days']
train,test,val=trainTestValSplit(ohe_df)
trainX,trainY=dfXYSplit(train,targetIndex,targetname)
testX,y_test=dfXYSplit(test,targetIndex,targetname)
valX,valY=dfXYSplit(val,targetIndex,targetname)
model.fit(trainX,trainY)
y_predict=model.predict(testX)
valPredY=model.predict(valX)

subtotal=pd.DataFrame()
subtotal['real_7days']=np.expm1(y_test[y_test.columns[0]].reset_index(drop=True))
subtotal['pred_7days']=np.expm1(np.array(y_predict))[:,0]
subtotal['real_14days']=np.expm1(y_test[y_test.columns[1]].reset_index(drop=True))
subtotal['pred_14days']=np.expm1(np.array(y_predict))[:,1]
subtotal['real_30days']=np.expm1(y_test[y_test.columns[2]].reset_index(drop=True))
subtotal['pred_30days']=np.expm1(np.array(y_predict))[:,2]
subtotal['real_60days']=np.expm1(y_test[y_test.columns[3]].reset_index(drop=True))
subtotal['pred_60days']=np.expm1(np.array(y_predict))[:,3]
subtotal['real_180days']=np.expm1(y_test[y_test.columns[4]].reset_index(drop=True))
subtotal['pred_180days']=np.expm1(np.array(y_predict))[:,4]
timelist=[7,14,30,60,180]
for i  in timelist:
    teststr=f'real_{i}days'
    predstr=f'pred_{i}days'
    test=subtotal[teststr]
    predict=subtotal[predstr]
    print(i,"_days_test_mean squared error: %.2f"
          % mean_squared_error(test,predict))
    print(i,"_days_test_mean_absolute_error: %.2f"
          % mean_absolute_error(test,predict))
    print(i,"_days_test_r2_score: %.2f"
          % r2_score(test,predict))
subtotal.head()

In [ ]:
from sklearn.externals import joblib
model = DecisionTreeRegressor(random_state=0,max_depth=20,min_samples_leaf=3,min_samples_split=8)
targetname=['sales_7days','sales_14days','sales_30days','sales_60days','sales_180days']
train,test,val=trainTestValSplit(ohe_df)
trainX,trainY=dfXYSplit(train,targetIndex,targetname)
testX,y_test=dfXYSplit(test,targetIndex,targetname)
valX,valY=dfXYSplit(val,targetIndex,targetname)
model.fit(trainX,trainY)
y_predict=model.predict(testX)
valPredY=model.predict(valX)
joblib.dump(model, 'saved_dtmodel.pkl') 


subtotal=pd.DataFrame()
subtotal['real_7days']=np.expm1(y_test[y_test.columns[0]].reset_index(drop=True))
subtotal['pred_7days']=np.expm1(np.array(y_predict))[:,0]
subtotal['real_14days']=np.expm1(y_test[y_test.columns[1]].reset_index(drop=True))
subtotal['pred_14days']=np.expm1(np.array(y_predict))[:,1]
subtotal['real_30days']=np.expm1(y_test[y_test.columns[2]].reset_index(drop=True))
subtotal['pred_30days']=np.expm1(np.array(y_predict))[:,2]
subtotal['real_60days']=np.expm1(y_test[y_test.columns[3]].reset_index(drop=True))
subtotal['pred_60days']=np.expm1(np.array(y_predict))[:,3]
subtotal['real_180days']=np.expm1(y_test[y_test.columns[4]].reset_index(drop=True))
subtotal['pred_180days']=np.expm1(np.array(y_predict))[:,4]
timelist=[7,14,30,60,180]
for i  in timelist:
    teststr=f'real_{i}days'
    predstr=f'pred_{i}days'
    test=subtotal[teststr]
    predict=subtotal[predstr]
    print(i,"_days_test_mean squared error: %.2f"
          % mean_squared_error(test,predict))
    print(i,"_days_test_mean_absolute_error: %.2f"
          % mean_absolute_error(test,predict))
    print(i,"_days_test_r2_score: %.2f"
          % r2_score(test,predict))
subtotal.head()

### XGBoost_Final Output

In [ ]:
import os
from sklearn.externals import joblib

from sklearn.multioutput import MultiOutputRegressor
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror')).fit(x_train, y_train)
targetname=['sales_7days','sales_14days','sales_30days','sales_60days','sales_180days']
train,test,val=trainTestValSplit(ohe_df)
trainX,trainY=dfXYSplit(train,targetIndex,targetname)
testX,y_test=dfXYSplit(test,targetIndex,targetname)
valX,valY=dfXYSplit(val,targetIndex,targetname)
model.fit(trainX,trainY)
y_predict=model.predict(testX)
valPredY=model.predict(valX)
joblib.dump(model, 'saved_xgbmodel_noparam.pkl') 

subtotal=pd.DataFrame()
subtotal['real_7days']=np.expm1(y_test[y_test.columns[0]].reset_index(drop=True))
subtotal['pred_7days']=np.expm1(np.array(y_predict))[:,0]
subtotal['real_14days']=np.expm1(y_test[y_test.columns[1]].reset_index(drop=True))
subtotal['pred_14days']=np.expm1(np.array(y_predict))[:,1]
subtotal['real_30days']=np.expm1(y_test[y_test.columns[2]].reset_index(drop=True))
subtotal['pred_30days']=np.expm1(np.array(y_predict))[:,2]
subtotal['real_60days']=np.expm1(y_test[y_test.columns[3]].reset_index(drop=True))
subtotal['pred_60days']=np.expm1(np.array(y_predict))[:,3]
subtotal['real_180days']=np.expm1(y_test[y_test.columns[4]].reset_index(drop=True))
subtotal['pred_180days']=np.expm1(np.array(y_predict))[:,4]
timelist=[7,14,30,60,180]
for i  in timelist:
    teststr=f'real_{i}days'
    predstr=f'pred_{i}days'
    test=subtotal[teststr]
    predict=subtotal[predstr]
    print(i,"_days_test_mean squared error: %.2f"
          % mean_squared_error(test,predict))
    print(i,"_days_test_mean_absolute_error: %.2f"
          % mean_absolute_error(test,predict))
    print(i,"_days_test_r2_score: %.2f"
          % r2_score(test,predict))
subtotal.head()

In [26]:
from sklearn.externals import joblib
# Output a pickle file for the model
import os
import pickle
from sklearn.multioutput import MultiOutputRegressor
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
targetname=['sales_7days','sales_14days','sales_30days','sales_60days','sales_180days']
train,test,val=trainTestValSplit(ohe_df)
trainX,trainY=dfXYSplit(train,targetIndex,targetname)
testX,y_test=dfXYSplit(test,targetIndex,targetname)
valX,valY=dfXYSplit(val,targetIndex,targetname)
xgbmodel = MultiOutputRegressor(xgb.XGBRegressor(max_depth=10,n_estimators=150)).fit(trainX,trainY)
y_predict=xgbmodel.predict(testX)
valPredY=xgbmodel.predict(valX)
xgbmodel.save_model('xgb.model')
pks = pickle.dumps(xgbmodel)
pickle.dump(xgbmodel, open("pima.pickle.dat", "wb"))
#joblib.dump(model, 'saved_xgbmodel_params.pkl') 

subtotal=pd.DataFrame()
subtotal['real_7days']=np.expm1(y_test[y_test.columns[0]].reset_index(drop=True))
subtotal['pred_7days']=np.expm1(np.array(y_predict))[:,0]
subtotal['real_14days']=np.expm1(y_test[y_test.columns[1]].reset_index(drop=True))
subtotal['pred_14days']=np.expm1(np.array(y_predict))[:,1]
subtotal['real_30days']=np.expm1(y_test[y_test.columns[2]].reset_index(drop=True))
subtotal['pred_30days']=np.expm1(np.array(y_predict))[:,2]
subtotal['real_60days']=np.expm1(y_test[y_test.columns[3]].reset_index(drop=True))
subtotal['pred_60days']=np.expm1(np.array(y_predict))[:,3]
subtotal['real_180days']=np.expm1(y_test[y_test.columns[4]].reset_index(drop=True))
subtotal['pred_180days']=np.expm1(np.array(y_predict))[:,4]
timelist=[7,14,30,60,180]
for i  in timelist:
    teststr=f'real_{i}days'
    predstr=f'pred_{i}days'
    test=subtotal[teststr]
    predict=subtotal[predstr]
    print(i,"_days_test_mean squared error: %.2f"
          % mean_squared_error(test,predict))
    print(i,"_days_test_mean_absolute_error: %.2f"
          % mean_absolute_error(test,predict))
    print(i,"_days_test_r2_score: %.2f"
          % r2_score(test,predict))
subtotal.head()

KeyboardInterrupt: 

In [ ]:
def trainTestValSplit(df):
    # 將資料切分為訓練(80%),測試(10%)和驗證(10%)三份。
    train=df.sample(frac=0.8,random_state=0)
    test=df.drop(train.index )
    val=test.sample(frac=0.5,random_state=0)
    test=test.drop(val.index)
    return train,test,val

def dfXYSplit(df,targetIndex,targetName):
    # 將特徵和目標變數切成兩份資料。
    
    dfX=df.drop(columns=targetIndex)
    dfY=df[targetName]
    
    return dfX,dfY

In [ ]:
from sklearn.externals import joblib
import os
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb

dt_model = joblib.load('saved_dtmodel.pkl') 
#bst3 = pickle.loads('saved_xgbmodel_params.pkl')
